# サンタンデール商品推奨のデータ解析

## データ準備

### ライブラリのインポート

In [ ]:
""" ライブラリのインポート
"""
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

### データの読み込み

In [ ]:
""" 学習データとテストデータの読み込み
"""
train_reader = pd.read_csv('./input/santander-product-recommendation/train_ver2.csv', chunksize=1000)
train = pd.concat((r for r in train_reader), ignore_index=True)

test_reader = pd.read_csv('./input/santander-product-recommendation/test_ver2.csv', chunksize=1000)
test = pd.concat((r for r in test_reader), ignore_index=True)

#train = pd.read_csv('./input/santander-product-recommendation/train_ver2.csv', low_memory=False)
#test = pd.read_csv('./input/santander-product-recommendation/test_ver2.csv')

## データ確認

train： 学習用データ  
test： 評価用データ  

### 欠損データの数を確認

In [ ]:
print('学習データの全体データ数')
print(len(train))
print('学習データの空データ率')
print(train.isnull().sum()/len(train))

In [ ]:
print('テストデータの全体データ数')
print(len(test))
print('テストデータの空データ率')
print(test.isnull().sum()/len(test))

In [ ]:
print(train.describe())

### 質データを数値データに変換

In [ ]:
""" 性別を男性:0、女性:1に変換
"""
#train['Sex'].fillna(('male'), inplace=True)
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1}).astype(int)

""" 乗船地を'S','C','Q'を0,1,2に変換
"""
train['Embarked'].fillna(('S'), inplace=True)
train['Embarked'] = train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)


### 各変数の相関を確認

生存と性別は関連がある。（女性の生存率が0.54と高い）


In [ ]:
""" 全体ヒートマップ
"""
sns.heatmap(train.dropna().corr())
print(train.dropna().corr())

In [ ]:
""" 女性のヒートマップ
"""
sns.heatmap(train[train['Sex'] == 1].corr())
print(train[train['Sex'] == 1].corr())

## ここで一度仮説を設定

ここで一度、生存と他のデータとの関係を考える
1. おそらく女性の生存率は高いだろう
2. おそらく子供の生存率は高いだろう
3. おそらく老人の生存率は高いだろう
4. おそらく金持ち（客室の等級が高い）は生存率は高いだろう
5. おそらく単身の男性は生存率が低いだろう
6. 

## 生存者・死亡者比
全データ900に対して、死亡が約550名、生存が約350名程度で、11:7の割合になっている。

In [ ]:
sns.countplot(data=train, x='Survived')
plt.xlabel('Survived (1:Survived, 0:Dead)')
plt.grid(True)

---

## 各パラメータと生死の関係を可視化

### 客室クラス 比較

客室クラスが良いほど優先的に救助はされているが、それほど顕著でもないなぁ （◎仮説1）

In [ ]:
sns.countplot(data=train, x='Pclass', hue='Survived')
plt.xlabel('Cabin Class (Pclass)')
plt.grid(True)

### 男女数 比較

明らかに女性は優先して救助されている（◎仮説1）

In [ ]:
sns.countplot(data=train, x='Sex', hue='Survived')
plt.xlabel('Sex')
plt.grid(True)

### 年齢の分布

5歳以下の乳児・幼児は明らかに優先されている。（◎仮説2）  
詳細に分析しないとわからないが、15歳くらいまでは優先されている???  
意外と老人は優先されていない。（✗仮説3）  



In [ ]:
sns.histplot(data=train, x='Age', binwidth=10, hue='Survived', multiple="dodge")
plt.xlabel('Age')
plt.grid(True)

In [ ]:
sns.histplot(data=train, x='Age', binwidth=5, hue='Survived', multiple="dodge")
plt.xlabel('Age')
plt.grid(True)

In [ ]:
sns.histplot(data=train[train['Sex'] == 1], x='Age', binwidth=5, hue='Survived', multiple="dodge")
plt.xlabel('Age')
plt.grid(True)

In [ ]:
sns.histplot(data=train[train['Sex'] == 1], x='Age', binwidth=5, hue='Survived', multiple="dodge")
plt.xlabel('Age')
plt.grid(True)

In [ ]:
sns.histplot(data=train[train['Sex'] == 1], x='Age', binwidth=5, hue='Survived', multiple="dodge")
plt.xlabel('Age')
plt.grid(True)

In [ ]:
sns.histplot(data=train[train['Sex'] == 0], x='Age', binwidth=5, hue='Survived', multiple="dodge")
plt.xlabel('Age')
plt.grid(True)

### 兄弟、配偶者の数

In [ ]:
sns.countplot(data=train, x='SibSp', hue='Survived')
plt.xlabel('SibSp')
plt.grid(True)

### 両親、子供の数

In [ ]:
sns.countplot(data=train, x='Parch', hue='Survived')
plt.xlabel('Parch')
plt.grid(True)

### 乗船地 比較

In [ ]:
sns.countplot(data=train, x='Embarked', hue='Survived')
plt.xlabel('Embarked')
plt.grid(True)

### チケット料金 分布

In [ ]:
sns.histplot(data=train, x='Fare', binwidth=50, hue='Survived', multiple='dodge')
plt.xlabel('Fare')
plt.grid(True)

In [ ]:
sns.histplot(data=train[train['Fare'] > 50], x='Fare', binwidth=50, hue='Survived', multiple='dodge')
plt.xlabel('Fare')
plt.grid(True)

50ドル以上料金を払っている人は生存率が高い、料金が50ドル以下だと生存率が低い（◎仮説4）

### 家族数を作る

In [ ]:
train['Fsize'] = train['Parch'] + train['SibSp'] + 1
print(train[train['Fsize'] > 5])

In [ ]:
sns.countplot(data=data, x='Fsize', hue='Survived')
plt.xlabel('Fsize')
plt.grid(True)

## 欠損データを補完 データのクリーニング

In [ ]:
""" 料金は平均値で補完
"""
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

""" 年齢と客室クラスは相関があるので、クラス毎の平均で補完
"""
data['Age'] = data.groupby('Pclass').transform(lambda d: d.fillna(d.mean()))['Age']